# Orchestraining training and deployment of scikit-learn model with Kubeflow Pipelines and Cloud AI Platform. 

In this lab you develop the KFP pipeline that orchestrates BigQuery and Cloud AI Platform services to train and deploy a **scikit-learn** model. The lab uses the [Covertype Dat Set](../datasets/covertype/README.md). The model is a multi-class classification model that predicts the type of forest cover from cartographic data. 

The source data is in BigQuery. The pipeline uses BigQuery to prepare training and evaluation splits, AI Platform Training to run a custom container with data preprocessing and training code, and AI Platform Prediction as a deployment target. The below diagram represents the workflow orchestrated by the pipeline.

![Training pipeline](../images/kfp-caip.png)



In [1]:
import kfp
import os
import uuid
import time
import tempfile

import pandas as pd

from google.cloud import bigquery
from jinja2 import Template
from kfp.components import func_to_container_op
from typing import NamedTuple


from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

/root/miniconda3/envs/tfx/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Configure environment settings
Make sure to update the constants to reflect your environment settings.

In [2]:
PROJECT_ID = 'mlops-workshop'
DATASET_LOCATION = 'US'
CLUSTER_NAME = 'mlops-workshop-cluster'
CLUSTER_ZONE = 'us-central1-a'
REGION = 'us-central1'
DATASET_ID = 'lab_12'
SOURCE_TABLE_ID = 'covertype'
TRAINING_TABLE_ID = 'training_split'
VALIDATION_TABLE_ID = 'validation_split'
TESTING_TABLE_ID = 'testing_split'
LAB_GCS_BUCKET='gs://mlops-workshop-lab-12'
TRAINING_FILE_PATH = LAB_GCS_BUCKET + '/datasets/covertype_training.csv'
VALIDATION_FILE_PATH = LAB_GCS_BUCKET + '/datasets/covertype_validation.csv'
TESTING_FILE_PATH = LAB_GCS_BUCKET + '/datasets/covertype_testing.csv'
COMPONENT_URL_SEARCH_PREFIX = 'https://raw.githubusercontent.com/kubeflow/pipelines/0.1.36/components/gcp/'

## Experimentation

### Exploring the source dataset 
Use BigQuery Python client library to query the data.

In [ ]:
client = bigquery.Client(project=PROJECT_ID, location=DATASET_LOCATION)

Read and display 100 rows from the source table.

In [ ]:
query_template = """
SELECT *
FROM `{{ source_table }}`
LIMIT 10
"""

query = Template(query_template).render(
    source_table='{}.{}.{}'.format(PROJECT_ID, DATASET_ID, SOURCE_TABLE_ID))
df = client.query(query).to_dataframe()
df

In [ ]:
query_template = """
SELECT count(*)
FROM `{{ source_table }}`
"""

query = Template(query_template).render(
    source_table='{}.{}.{}'.format(PROJECT_ID, DATASET_ID, SOURCE_TABLE_ID))
df = client.query(query).to_dataframe()
number_of_rows_in_full_dataset = df.iloc[0,0]
print(number_of_rows_in_full_dataset)

### Create the training, validation and testing splits
#### Define the sampling query template

In [ ]:
sampling_query_template = """
SELECT *
FROM 
  `{{ source_table }}` AS cover
WHERE 
  MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), {{ num_lots }}) in {{ lots_to_select }}
"""

#### Configure query job settings

In [ ]:
job_config = bigquery.QueryJobConfig()
job_config.create_disposition = bigquery.job.CreateDisposition.CREATE_IF_NEEDED
job_config.write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
dataset_ref = client.dataset(DATASET_ID)

#### Create the training split

In [ ]:
query = Template(sampling_query_template).render(
    source_table='{}.{}.{}'.format(PROJECT_ID, DATASET_ID, SOURCE_TABLE_ID),
    num_lots=10,
    lots_to_select='(1, 2, 3)')

job_config.destination = dataset_ref.table(TRAINING_TABLE_ID)
client.query(query, job_config).result()

#### Extract the training split to GCS

In [ ]:
client.extract_table(dataset_ref.table(TRAINING_TABLE_ID), TRAINING_FILE_PATH).result()

In [ ]:
!gsutil cat -r 0-500 {TRAINING_FILE_PATH}

#### Create the validation split

In [ ]:
query = Template(sampling_query_template).render(
    source_table='{}.{}.{}'.format(PROJECT_ID, DATASET_ID, SOURCE_TABLE_ID),
    num_lots=10,
    lots_to_select='(8)')

job_config.destination = dataset_ref.table(VALIDATION_TABLE_ID)
client.query(query, job_config).result()

#### Extract the validation split to GCS

In [ ]:
client.extract_table(dataset_ref.table(VALIDATION_TABLE_ID), VALIDATION_FILE_PATH).result()

#### Create the testing split

In [ ]:
query = Template(sampling_query_template).render(
    source_table='{}.{}.{}'.format(PROJECT_ID, DATASET_ID, SOURCE_TABLE_ID),
    num_lots=10,
    lots_to_select='(9)')

job_config.destination = dataset_ref.table(TESTING_TABLE_ID)
client.query(query, job_config).result()

#### Extract the testing split to GCS

In [ ]:
client.extract_table(dataset_ref.table(TESTING_TABLE_ID), TESTING_FILE_PATH).result()

### Developing the training script

#### Configure training pipeline

In [ ]:
numeric_features = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points']
categorical_features = ['Wilderness_Area', 'Soil_Type']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features) 
    ])

alpha = 0.0001
max_iter = 1000

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(alpha=alpha, max_iter=max_iter, loss='log'))
])

#### Train and evaluate

In [ ]:
df_train = pd.read_csv(TRAINING_FILE_PATH)
df_validation = pd.read_csv(VALIDATION_FILE_PATH)

X_train = df_train.drop('Cover_Type', axis=1)
y_train = df_train['Cover_Type']
X_validation = df_validation.drop('Cover_Type', axis=1)
y_validation = df_validation['Cover_Type']

pipeline.fit(X_train, y_train)
pipeline.score(X_validation, y_validation)

### Hyperparameter tuning
Since the training run on this dataset is computationally expensive you can benefit from running a distributed hyperparameter tuning job on AI Platform Training.

#### Prepare a hyperparameter tuning script

In [ ]:
TRAINING_APP_FOLDER = 'training_app'
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)

In [ ]:
%%writefile {TRAINING_APP_FOLDER}/train.py

import os
import subprocess
import sys

import fire
import joblib
import numpy as np
import pandas as pd

import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def train_evaluate(job_dir, training_dataset_path, validation_dataset_path, alpha, max_iter, dataset_location='US'):
    
  df_train = pd.read_csv(training_dataset_path)
  df_validation = pd.read_csv(validation_dataset_path)

  numeric_features = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points']
    
  categorical_features = ['Wilderness_Area', 'Soil_Type']

  preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features) 
    ])

  pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(alpha=alpha, max_iter=max_iter, loss='log'))
  ])

  print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))
  X_train = df_train.drop('Cover_Type', axis=1)
  y_train = df_train['Cover_Type']
  X_validation = df_validation.drop('Cover_Type', axis=1)
  y_validation = df_validation['Cover_Type']
  pipeline.fit(X_train, y_train)
  accuracy = pipeline.score(X_validation, y_validation)
  print('Finished training. Model accuracy: {}'.format(accuracy))
    
  # Log it with hypertune
  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag='accuracy',
    metric_value=accuracy
    )

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=pipeline, filename=model_filename)
  gcs_model_path = "{}/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  print("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  fire.Fire(train_evaluate)

#### Package the script 

In [ ]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

In [ ]:
IMAGE_NAME='covertype_trainer'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)

!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

#### Create hyperparameter configuration file

In [ ]:
%%writefile {TRAINING_APP_FOLDER}/hptuning_config.yaml

trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 10
    maxParallelTrials: 3
    hyperparameterMetricTag: accuracy
    enableTrialEarlyStopping: TRUE 
    params:
    - parameterName: max_iter
      type: DISCRETE
      discreteValues: [
          500,
          1000
          ]
    - parameterName: alpha
      type: DOUBLE
      minValue:  0.00001
      maxValue:  0.01
      scaleType: UNIT_LINEAR_SCALE

#### Submit hyperparameter tuning job

In [ ]:
IMAGE_NAME='covertype_trainer'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(LAB_GCS_BUCKET, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$LAB_GCS_BUCKET/$JOB_NAME \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
--config $TRAINING_APP_FOLDER/hptuning_config.yaml \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH 

In [ ]:
!gcloud ai-platform jobs describe $JOB_NAME

In [ ]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

## Create the training pipeline

In [3]:
sampling_query_template = """
SELECT *
FROM 
  `{{ source_table }}` AS cover
WHERE 
  MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), {{ num_lots }}) in {{ lots_to_select }}
"""

In [4]:
query = Template(sampling_query_template).render(
    source_table='{}.{}.{}'.format(PROJECT_ID, DATASET_ID, SOURCE_TABLE_ID),
    num_lots=10,
    lots_to_select='(8)')

In [5]:
component_store = kfp.components.ComponentStore(
    local_search_paths=None,
    url_search_prefixes=[COMPONENT_URL_SEARCH_PREFIX])
    
bigquery_query_op = component_store.load_component('bigquery/query')

In [6]:
output_gcs_path = '{}/{}'.format(LAB_GCS_BUCKET, 'sample_data')

In [15]:
from kfp.dsl.types import GCPProjectID

@kfp.dsl.pipeline(
    name='Covertype Classifier Training',
    description='The pipeline training and deploying the Covertype classifierpipeline_yaml'
)
def covertype_train(
    project_id:GCPProjectID =PROJECT_ID,
    query:str =query,
    table_id:str =TRAINING_TABLE_ID,
    dataset_id:str =DATASET_ID,
    dataset_location:str =DATASET_LOCATION,
    output_gcs_path:str =output_gcs_path
    ):
    
    
    
    sample_data = bigquery_query_op(
        query=query,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id=table_id,
        output_gcs_path=output_gcs_path,
        dataset_location=dataset_location
        )
    
    from kfp.gcp import use_gcp_secret
    kfp.dsl.get_pipeline_conf().add_op_transformer(use_gcp_secret('user-gcp-sa'))


In [16]:
pipeline_yaml = 'covertype_training.yaml'
kfp.compiler.Compiler().compile(covertype_train, pipeline_yaml)

In [17]:
pipeline_name = 'covertype_training_pipeline'
client = kfp.Client()

pipelines = [pipeline for pipeline in client.list_pipelines(page_size=100).pipelines if pipeline.name == pipeline_name]

if pipelines:
    print("Pipeline with this name already exists")
    pipeline_ref = pipelines[0]
    
else:
    pipeline_ref = client.upload_pipeline(pipeline_yaml, pipeline_name)